In [1]:
import pandas as pd
import argparse
import logging
import json
import os
        import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
# Define constants

BELLINZONA_UST = 2101
BLENIO_UST = 2102
LEVENTINA_UST = 2103
LOCARNO_UST = 2104
LUGANO_UST = 2105
MENDRISIO_UST = 2106
RIVIERA_UST = 2107
VALLEMAGGIA = 2108

### ODIS ### 
# AVG_RES_LIGHT_DENSITY = 8.3 # kWh/m2y ODIS
# AVG_RES_APP =  1400 # kWh/y ODIS, 

### ROHDATEN Tarife 2020 ELCOM ###
# H1 - 1600 kWh/an: logement de 2 pièces avec cuisinière électrique
# H2 - 2500 kWh/an: logement de 4 pièces avec cuisinière électrique; 
# H4 - 4500 kWh/an: logement de 5 pièces avec cuisinière électrique et sèche-linge (sans chauffe-eau électrique) 
# H8 - 7500 kWh/an: grand logement en propriété, avec large utilisation de l'électricité 
# C1 - 8000 kWh/an: très petite entreprise, puissance max.: 8 kW 
# C2 - 30 000 kWh/an: petite entreprise, puissance max.: 15 kW 
# C3 - 150 000 kWh/an: entreprise moyenne, puissance max.: 50 kW
# C4 - 500 000 kWh/an: grande entreprise, puissance max.: 150 kW, courant basse tension / C5 - courant moyenne tension, propre station de transformation 
# C6 - 1 500 000 kWh/an: grande entreprise, puissance max.: 400 kW, courant moyenne tension, propre station de transformation 
# C7 - 7 500 000 kWh/an: grande entreprise, puissance max.: 1630 kW, courant moyenne tension, propre station de transformation 

### Consumi di energia (in gigawattora), secondo la destinazione e il vettore energetico, in Ticino, nel 2020
S1_AGR_elec = 64000 # kWh/y
S2_IND_elec = 160000 #kWh/y
S3_SER_elec = 25000 #kWh/y

### https://pubdb.bfe.admin.ch/it/publication/download/10559 ###
MFH_2pp_elec = 2190 # kWh/y =  apps and light
MFH_1pp_elec_diff = 458.5 # kWh/y =  apps and light
SFH_4pp_elec = 4048 # kWh/y =  apps and light
SFH_1pp_elec_diff = 593.5 # kWh/y =  apps and light

AVG_DHW_PP = 40 # L/DAY/PERSON ODIS
AVG_M2_PP = 49.1 #TI - Superficie media per occupante, per Cantone, 2021, https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/superficie-persona.html
SUPERFICIE_MEDIA = 97.8 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/dimensioni.html

MFH_PP = 2.1 #https://www.bfs.admin.ch/bfs/it/home/statistiche/costruzioni-abitazioni/abitazioni/condizioni-abitazione/densita-utilizzazione.html
SFH_PP = 2.7

GKLAS = [1110,1121,1122,1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1241, 1242, 1271, 1274, 1275, 1276, 1277, 1278, 1231, 1252, 1273]

GKLAS_SFH = 1110
GKLAS_DFH = 1121
GKLAS_MFH = 1122
GKLAS_RES = [GKLAS_SFH, GKLAS_DFH, GKLAS_MFH]

GKLAS_HTG = [1110, 1121, 1122, 1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1231]
GKLAS_NOHTG = [1241, 1242, 1252, 1271, 1273, 1274, 1275, 1276, 1277, 1278]

GBAUP = [8011,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022,8023]
GSTAT = 1004 # existing
   
pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'

#SELECT CANTON
#CANTON = "Genève"
CANTON = "Ticino"

# Read the shp file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
SWISSTOPO_DISTRICT_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_BEZIRKSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html
SWISSTOPO_CANTON_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html. NOTE: The actual shapefile (.shp) is useless without the companion files: .dbf, .shx, .prj etc..
### SWISSTOPO_CANTON_FILE = figureout how to get the municipal boundaries https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill

CANTON_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_CANTON_FILE
DISTRICT_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_DISTRICT_FILE

CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

In [3]:
# Calculate the electric demand for agriculture, industry, and services - top down - CODE WORKS

### Consumi di energia (in gigawattora), secondo la destinazione e il vettore energetico, in Ticino, nel 2020
S1_AGR_elec = 61640 # kWh/y = consumo "Altro (cantieri, agricoltura ecc.)" 2020 / Aziende primario 2020
S2_IND_elec = 153650 #kWh/y consumo "Artigianato e industria"/ Aziende secondario 2020
S3_SER_elec = 22180 #kWh/y consumo "Commercio e servizi" /Aziende terzario 2020

fileDir = DATA_DIRECTORY / "support_data" 
data_s123 = pd.read_csv(fileDir/"S1_S2_S3.csv" , header=0, index_col=0)

S123 = pd.DataFrame()

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

for dis_num in UST_district:
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    dis_S1_tot_elec = data_s123.loc[DISTRICT, "S1_Aziende"]*S1_AGR_elec
    dis_S2_tot_elec = data_s123.loc[DISTRICT, "S2_Aziende"]*S2_IND_elec
    dis_S3_tot_elec = data_s123.loc[DISTRICT, "S3_Aziende"]*S3_SER_elec
    
    tot_elec = pd.DataFrame(np.array([[dis_S1_tot_elec], [dis_S2_tot_elec], [dis_S3_tot_elec]]),  columns = [DISTRICT], index = [ "S1_AGR", "S2_IND", "S3_SER"]) 
    
    S123 = pd.concat([S123,tot_elec], axis = 1) #,ignore_index=True,
S123["Total"] = S123.sum(axis=1)
S123

,Bellinzona,Blenio,Leventina,Locarno,Lugano,Mendrisio,Riviera,Vallemaggia,Total
S1_AGR,10725360,7643360,7088600,12081440,15718200,11465040,2835440,4438080,71995520
S2_IND,94494750,16286900,23047500,155032850,348478200,156569350,25659550,24430350,843999450
S3_SER,86080580,7119780,11666680,118707360,362421200,123054640,12709140,8716740,730476120


### RESIDENTIAL

In [204]:
# PER DISTRICT - Getting the number of households according to ERA (EWID) - WORKS
### TO-DO
# 0. when missing raw-gdf files use DATA -> DONE
# 1. check for and remove duplicates -> there are egid duplicates but each have different ewids, for the purpose of elec keep all, but need to write code that combines the ewids for each egid. For heating work with EGID since the garea is duplicated
# 2. add fillers for garea, gbaup, gastw and combine with htg calc
# 3. to_csv

import ast

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]

UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

TI_elec = pd.DataFrame()
district_res_elec = pd.DataFrame()
ewid_count = pd.DataFrame()
#dis_num = 2101

AVG_HH_ELEC_TI = 2741 # kWh/y USTAT 2020 = tot apparecchi/tot abitazioni ewid 
UNOCC = 0.0283 # should be around 0.0283 for TI

for dis_num in UST_district:
    communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]

    for com_num in communes:
        #print("comune: ", com_num)
        fileloc =  CANTON_CSV_DIRECTORY

        filename_gdf = "raw-gdf-"+str(com_num)+".csv"
        filename_data = "DATA_"+str(com_num)+".csv"
        
        if pathlib.Path(fileloc/filename_gdf).exists():
            data_res_dirty = pd.read_csv(fileloc/filename_gdf , header=0, index_col=0)
            data_res = data_res_dirty.drop_duplicates(keep="first")
        elif pathlib.Path(fileloc/filename_data).exists():
            data_res_dirty = pd.read_csv(fileloc/filename_data , header=0, index_col=0)
            data_res = data_res_dirty.drop_duplicates(keep="first")
        
        data_res = data_res.loc[data_res["gstat"] == 1004]
        data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
        data_res = data_res[REA_CODES_DESIRED_LC]


        data_res["gklas_fill"] =  data_res["gklas"]
        data_res = data_res.reset_index()

        for g in range(len(data_res)):
            if (data_res.at[g,"gkat"] == 1020):
                data_res.loc[g,"gklas_fill"] = 1110
            elif (data_res.at[g,"gkat"] == 1030):
                data_res.loc[g,"gklas_fill"] = 1122 # 63% of known 1040

        data_res["ewid"] = data_res["ewid"].fillna(value = "['no']")

        for g in range(len(data_res)):
            if (data_res.at[g,"ewid"]=="['no']"):
                if (data_res.at[g,"gklas_fill"] == 1110):
                    data_res.at[g,"ewid"] = "['1']"
                elif (data_res.at[g,"gklas_fill"] == 1121):
                    data_res.at[g,"ewid"] = "['1', '2']"
                elif (data_res.at[g,"gklas_fill"] == 1122):
                    data_res.at[g,"ewid"] = "['1', '2', '3', '4', '5', '6', '7']" # GET real median
            else:
                pass

        for g in range(len(data_res)):
            ewid_count.at[g,"ewid"] = ast.literal_eval(data_res.at[g,"ewid"])      

        data_res["ewid_len"] = ewid_count.ewid.apply(lambda x: len(x))

        district_res_elec = pd.concat([district_res_elec,data_res])
        
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    d_hh = (1-UNOCC)*(district_res_elec["ewid_len"].sum()) #
    d_elec = d_hh*AVG_HH_ELEC_TI
    d = pd.DataFrame([d_hh, d_elec], index=["hh","res_elec"], columns=[DISTRICT])

    TI_elec = pd.concat([TI_elec,d], axis=1)
    district_res_elec = district_res_elec[0:0]

TI_elec["TI"] = TI_elec.sum(axis=1)
TI_elec

,Bellinzona,Blenio,Leventina,Locarno,Lugano,Mendrisio,Riviera,Vallemaggia,TI
hh,3.288816e+04,6.942797e+03,9.396339e+03,5.741387e+04,9.589124e+04,3.033745e+04,6.240257e+03,6.692098e+03,2.458022e+05
res_elec,9.014644e+07,1.903021e+07,2.575537e+07,1.573714e+08,2.628379e+08,8.315494e+07,1.710455e+07,1.834304e+07,6.737438e+08


# TESTS

In [142]:
fileDir = DATA_DIRECTORY / "support_data" 
data_s123 = pd.read_csv(fileDir/"S1_S2_S3.csv" , header=0, index_col=0)
data_s123

,Totale__Imprese,S1_Imprese,S2_Imprese,S3_Imprese,Totale_Aziende,S1_Aziende,S2_Aziende,S3_Aziende
Mendrisio,6126,176,980,4970,6753,186,1019,5548
Lugano,17354,247,2203,14904,18863,255,2268,16340
Locarno,5885,189,970,4726,6557,196,1009,5352
Vallemaggia,566,68,153,345,624,72,159,393
Bellinzona,3835,168,582,3085,4670,174,615,3881
Riviera,675,47,155,473,786,46,167,573
Blenio,516,124,108,284,551,124,106,321
Leventina,680,111,126,443,791,115,150,526


### S1 -AGRICULTURE, S2 -INDUSTRY, S3 -SERVICES

In [15]:
# LOOP THROUGH ALL DISTRICT
#for dis_num in UST_district:

from operator import itemgetter

DICT = {"SFH":1110.0, "DFH":1121.0, "MFH":1122.0, "HABITAT_COMMUNAUTAIRE":1130.0, "HOTEL":1211.0, "HEBERGEMENT":1212.0, "OFFICE":1220.0, "COMMERCIAL":1230.0, "RESTO_BAR":1231.0, "TRANSP_STATIONS":1241.0, "GARAGE": 1242.0, 
        "INDUSTRIAL":1251.0, "RESERVOIRS":1252.0, "CULTURAL":1261.0, "MUSEUM_LIBRARY":1262.0, "ACADEMIC":1263.0, "HOSPITAL":1264.0, "SPORTS":1265.0, "AGRICULTURE":1271.0, "RELIGIOUS":1272.0, "ANCIENT":1273.0, "OTHER_PUBLIC":1274.0, "OUTSIDE": 1275.0,
        "ANIMALS":1276.0, "GREENHOUSE":1277.0, "AGRI_STORAGE":1278.0}

RES_code = itemgetter("SFH", "DFH", "MFH")(DICT)
AGR_code = itemgetter("AGRICULTURE")(DICT) #, "ANIMALS", "GREENHOUSE", "AGRI_STORAGE"
IND_code = itemgetter("INDUSTRIAL")(DICT)
SER_code = itemgetter("HABITAT_COMMUNAUTAIRE","HOTEL", "HEBERGEMENT", "OFFICE", "COMMERCIAL", "RESTO_BAR", "TRANSP_STATIONS","CULTURAL", "MUSEUM_LIBRARY", "ACADEMIC", "HOSPITAL", "SPORTS")(DICT)
OTHER_code = itemgetter("GARAGE","RESERVOIRS", "ANCIENT", "OUTSIDE", "RELIGIOUS","OTHER_PUBLIC")(DICT)

# SINGLE DISTRICT
dis_num = LUGANO_UST
DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
dis_filename = DISTRICT + ".csv"
dis_csv = CANTON_CSV_DIRECTORY / dis_filename
dis_rea = pd.read_csv(dis_csv, sep= ";", header=0, index_col=0).set_index("EGID")
dis_rea = dis_rea.drop(["index.1"], axis=1)

dr = dis_rea.drop(["GABBJ","GSTAT", "GBAUJ", "GBAUP","GWAERZW1","GENW1","GWAERDATW1", "GWAERDATH1", "GENH1", "GWAERZH1","GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1) #
dr = dr.reset_index()
#dr.dtypes

RES = dr.loc[dr["GKLAS_fill"].isin(RES_code)]
AGR = dr.loc[dr["GKLAS_fill"] == (AGR_code)]
IND = dr.loc[dr["GKLAS_fill"] == (IND_code)]
SER = dr.loc[dr["GKLAS_fill"].isin(SER_code)]
OTHER = dr.loc[dr["GKLAS_fill"].isin(OTHER_code)]

print(DISTRICT, "RES:", RES["EGID"].count() , "AGR:", AGR["EGID"].count(), "IND:", IND["EGID"].count() , "SER:", SER["EGID"].count(), "OTHER:", OTHER["EGID"].count())

Lugano RES: 37101 AGR: 5758 IND: 547 SER: 2231 OTHER: 9221


### RESIDENTIAL

In [131]:
# PER COMUNE -  Getting the number of households according to ERA (EWID) -> matches pretty well with this layer: https://map.geo.admin.ch/?layers=ch.are.wohnungsinventar-zweitwohnungsanteil
import ast

pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'
CANTON = "Ticino"
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]


#for com_num in communes:
com_num = 5269    
print("comune: ", com_num)
fileloc =  CANTON_CSV_DIRECTORY
# reading comune raw gdf file
filename = "raw-gdf-"+str(com_num)+".csv"
data_res = pd.read_csv(fileloc/filename , header=0, index_col=0)
data_res = data_res.loc[data_res["gstat"] == 1004]
data_res = data_res.loc[(data_res["gkat"] == 1020) | (data_res["gkat"] == 1030)]
data_res = data_res[REA_CODES_DESIRED_LC]


data_res["gklas_fill"] =  data_res["gklas"]
data_res = data_res.reset_index()

for g in range(len(data_res)):
    if (data_res.at[g,"gkat"] == 1020):
        data_res.loc[g,"gklas_fill"] = 1110
    elif (data_res.at[g,"gkat"] == 1030):
        data_res.loc[g,"gklas_fill"] = 1122 # 63% of known 1040
        
data_res["ewid"] = data_res["ewid"].fillna(value = "['no']")

for g in range(len(data_res)):
    if (data_res.at[g,"ewid"]=="['no']"):
        if (data_res.at[g,"gklas_fill"] == 1110):
            data_res.at[g,"ewid"] = "['1']"
        elif (data_res.at[g,"gklas_fill"] == 1121):
            data_res.at[g,"ewid"] = "['1', '2']"
        elif (data_res.at[g,"gklas_fill"] == 1122):
            data_res.at[g,"ewid"] = "['1', '2', '3', '4', '5', '6', '7']" # GET real median
    else:
        pass

for g in range(len(data_res)):
    data_res.at[g,"ewid"] = ast.literal_eval(data_res.at[g,"ewid"])      

data_res["ewid_len"] = data_res.ewid.apply(lambda x: len(x))

test = data_res[["gklas_fill","ewid","ewid_len"]]

print(test["ewid_len"].sum())


comune:  5269
1493


In [33]:
## PER COMUNE
REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2',"ewid"]
b = district_res_elec.loc[district_res_elec["ggdenr"] == 5009]# GGDENR Numéro OFS de la commune
b = b[REA_CODES_DESIRED_LC]
b = b.reset_index()
b.isna().sum()

index              0
egid               0
strname_deinr      0
ggdename           0
ggdenr             0
gexpdat            0
gdekt              0
egrid            204
gebnr              0
gkode              0
gkodn              0
gksce              0
gstat              0
gkat               0
gklas             65
gbauj            318
gbaup             86
gabbj            486
garea              0
gastw             54
gazzi            486
gebf             486
gwaerzh1         226
genh1            226
gwaersceh1       226
gwaerdath1         0
gwaerzh2         458
genh2            458
gwaersceh2       458
gwaerdath2         0
gwaerzw1         226
genw1            226
gwaerscew1       226
gwaerdatw1         0
gwaerzw2         366
genw2            366
gwaerscew2       366
gwaerdatw2         0
ewid             233
dtype: int64

In [36]:
# AGE GROUPBY - COUNT
b = district_res_elec.loc[district_res_elec["ggdenr"] == 5009]# GGDENR Numéro OFS de la commune

GKAT_group = b[["gkat", "gklas", "ewid"]]
GKAT_group = GKAT_group.groupby(["gkat", "gklas"]).count()
GKAT_group

ewid
gkat gklas       
1020 1110.0   205
     1121.0    34
     1122.0     7
1030 1110.0     1
     1121.0     1
1040 1211.0     1
1060 1220.0     0
     1242.0     0
     1251.0     0
     1252.0     0
     1263.0     0
     1271.0     1
     1272.0     0
     1274.0     0

In [17]:
# Calculating number of households and ppl, and elec consumption
for dis_num in UST_district:
    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    dis_filename = DISTRICT + ".csv"
    dis_csv = CANTON_CSV_DIRECTORY / dis_filename
    dis_rea = pd.read_csv(dis_csv, sep= ";", header=0, index_col=0).set_index("EGID")
    dis_rea = dis_rea.drop(["index.1"], axis=1)
    dr = dis_rea.drop(["GABBJ","GSTAT", "GBAUJ", "GBAUP","GWAERZW1","GENW1","GWAERDATW1", "GWAERDATH1", "GENH1", "GWAERZH1","GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1) #
    dr = dr.reset_index()
    
    dr["ppl"] = 2.4
    dr["elec"] = SFH_4pp_elec
    dr["num_hh"] = 1

for g in range(len(dr)):
    
    ### RESIDENTIAL
    if dr.at[g,"GKLAS_fill"] == 1110:
        dr.at[g,"ppl"] = min(0.93*SFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/AVG_M2_PP))
        dr.at[g,"elec"] = SFH_4pp_elec + (dr.at[g,"ppl"]-4)*SFH_1pp_elec_diff
        
    elif dr.at[g,"GKLAS_fill"] == 1121:
        dr.at[g,"num_hh"] = 2
        dr.at[g,"ppl"] = dr.at[g,"num_hh"]*(min(1.1*MFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/dr.at[g,"num_hh"]/AVG_M2_PP)))
        dr.at[g,"elec"] = SFH_4pp_elec + (dr.at[g,"ppl"]-4)*SFH_1pp_elec_diff
    
    elif dr.at[g,"GKLAS_fill"] == 1122:
#        if (dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]) <SUPERFICIE_MEDIA:
#            dr.at[g,"num_hh"] = dr.at[g,"GASTW_fill"]
#        else:
        dr.at[g,"num_hh"] = 0.6*(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"])/(1.5*SUPERFICIE_MEDIA)
        dr.at[g,"ppl"] = dr.at[g,"num_hh"]*min(1.1*MFH_PP,(dr.at[g,"GAREA_fill"]*dr.at[g,"GASTW_fill"]/dr.at[g,"num_hh"]/AVG_M2_PP))
        dr.at[g,"elec"] = MFH_2pp_elec + (dr.at[g,"ppl"]-2)*MFH_1pp_elec_diff    
    
    ### SERVICES
    else:
        dr.at[g,"ppl"] = 0
        dr.at[g,"elec"] = 0

In [18]:
dr_SFH = dr.loc[dr["GKLAS_fill"] == 1110]
print("SFH_median: ",round(dr_SFH["ppl"].median(),1), "SFH_mean: ",round(dr_SFH["ppl"].mean(),1))
        
dr_DFH = dr.loc[dr["GKLAS_fill"] == 1121]
print("DFH_median: ", round(dr_DFH["ppl"].median()/dr_DFH["num_hh"].median(),1), "DFH_mean: ",round(dr_DFH["ppl"].mean()/dr_DFH["num_hh"].median(),1))

dr_MFH = dr.loc[dr["GKLAS_fill"] == 1122]
print("MFH_median: ",round(dr_MFH["ppl"].median()/dr_MFH["num_hh"].median(),1), "MFH_mean: ",round(dr_MFH["ppl"].mean()/dr_MFH["num_hh"].median(),1))

SFH_median:  2.3 SFH_mean:  2.0
DFH_median:  2.3 DFH_mean:  2.1
MFH_median:  2.3 MFH_mean:  3.1


In [19]:
dr_hh = dr["num_hh"].sum()
dr_ppl = dr["ppl"].sum()
dr_elec = dr["elec"].sum()
print( DISTRICT," hh:",dr_hh, " ppl:",dr_ppl, " elec kWh:", dr_elec)

Vallemaggia  hh: 12358.69529652352  ppl: 13111.829279572177  elec kWh: 17332612.34920523


In [10]:
dr_SFH.head()

,EGID,GKODE,GKODN,GKAT,GKLAS,GAREA,GASTW,GGDENR,GGDENAME,GDEKT,...,k_DHW,k_fehh,ERA,SH,DHW,TOT_TH,P_tot,ppl,elec,num_hh
4,11167148,2681172.941,1130101.284,1020.0,1110.0,88.0,2.0,5304,Bosco/Gurin,TI,...,14.0,2000.0,158.4,22334.4,2217.6,24552.0,12.2760,2.511000,3164.278500,1.0
5,11167149,2681015.939,1130059.290,1020.0,1110.0,78.0,1.0,5304,Bosco/Gurin,TI,...,14.0,2000.0,70.2,10459.8,982.8,11442.6,5.7213,1.588595,2616.830957,1.0
6,11167150,2681017.620,1130069.060,1020.0,1110.0,61.0,2.0,5304,Bosco/Gurin,TI,...,14.0,2000.0,109.8,16360.2,1537.2,17897.4,8.9487,2.484725,3148.684318,1.0
7,11167151,2681007.470,1130012.490,1020.0,1110.0,42.0,2.0,5304,Bosco/Gurin,TI,...,14.0,2000.0,75.6,11264.4,1058.4,12322.8,6.1614,1.710794,2689.356415,1.0
8,11167152,2681037.420,1130003.260,1020.0,1110.0,82.0,1.0,5304,Bosco/Gurin,TI,...,14.0,2000.0,73.8,10996.2,1033.2,12029.4,6.0147,1.670061,2665.181263,1.0


In [11]:
dr_MFH.head()

,EGID,GKODE,GKODN,GKAT,GKLAS,GAREA,GASTW,GGDENR,GGDENAME,GDEKT,...,k_DHW,k_fehh,ERA,SH,DHW,TOT_TH,P_tot,ppl,elec,num_hh
19,11167163,2681205.77,1130089.12,1020.0,1122.0,167.0,4.0,5304,Bosco/Gurin,TI,...,21.0,2000.0,601.2,92584.8,12625.2,105210.0,52.60500,6.311166,4166.669448,2.732106
25,11167169,2681194.29,1130003.25,1020.0,1122.0,117.0,3.0,5304,Bosco/Gurin,TI,...,21.0,2000.0,315.9,47069.1,6633.9,53703.0,26.85150,3.316196,2793.476012,1.435583
30,11167174,2681187.26,1129972.41,1020.0,1122.0,78.0,2.0,5304,Bosco/Gurin,TI,...,21.0,2000.0,140.4,18392.4,2948.4,21340.8,10.67040,1.473865,1948.767117,0.638037
40,11167185,2681264.37,1129985.78,1020.0,1122.0,107.0,5.0,5304,Bosco/Gurin,TI,...,21.0,2000.0,481.5,48150.0,10111.5,58261.5,29.13075,5.054601,3590.534663,2.188139
44,11167189,2681330.13,1130132.27,1020.0,1122.0,108.0,3.0,5304,Bosco/Gurin,TI,...,21.0,2000.0,291.6,41115.6,6123.6,47239.2,23.61960,3.061104,2676.516319,1.325153


In [12]:
small = dr_MFH.loc[dr_MFH["GAREA_fill"] < SUPERFICIE_MEDIA]
big = dr_MFH.loc[dr_MFH["ppl"] > 40]
print(small, big)

           EGID        GKODE        GKODN    GKAT   GKLAS  GAREA  GASTW   
30     11167174  2681187.260  1129972.410  1020.0  1122.0   78.0    2.0  \
591   190929949  2684175.520  1127187.930  1020.0  1122.0   97.0    3.0   
1490   11167354  2689996.930  1133294.480  1030.0  1122.0   87.0    3.0   
1504   11167370  2690114.480  1133321.780  1020.0  1122.0   84.0    3.0   
1653   11167558  2689300.689  1129758.806  1020.0  1122.0   62.0    4.0   
2250  190802529  2689698.003  1129961.283  1030.0  1122.0   72.0    3.0   
3689   11167685  2695773.920  1125563.020  1020.0  1122.0   90.0    2.0   
3960   11168218  2698072.040  1122527.630  1020.0  1122.0   96.0    4.0   
4002   11168301  2697758.709  1121747.600  1020.0  1122.0   81.0    3.0   
4022   11168324  2697726.120  1121869.260  1020.0  1122.0   87.0    4.0   
4031   11168333  2697687.990  1121913.320  1020.0  1122.0   94.0    2.0   
4997  190212796  2696130.100  1123721.310  1020.0  1122.0   69.0    3.0   
7887   11168412  2689924.